# NASDAQ LSTM Hyperparameter Tuning

This notebook implements automated hyperparameter optimization for LSTM neural networks applied to NASDAQ time series prediction using Optuna framework.

## Dependencies and Imports

Essential libraries for data processing, machine learning, and hyperparameter optimization.

In [1]:
%%capture
%pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import optuna
import os
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Data Loading and Preprocessing

Load NASDAQ historical data and prepare it for time series modeling with normalization.

In [ ]:
df = pd.read_csv('data/HistoricalData.csv')
df.rename(columns={'Close/Last':'Close'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', ascending=True, inplace=True)
series = pd.to_numeric(df['Close'], errors='coerce').dropna().values.reshape(-1,1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(series)

## Sequence Creation Utility

Function to create input-output sequences for LSTM training with configurable lookback window.

In [ ]:
def make_sequences(arr, look_back):
    X, y = [], []
    for i in range(look_back, len(arr)):
        X.append(arr[i-look_back:i])
        y.append(arr[i])
    X = np.array(X)
    y = np.array(y)
    return X, y

## Model Training and Evaluation Function

Comprehensive function that builds, trains, and evaluates LSTM models with given hyperparameters.

In [ ]:
train_ratio = 0.8

def train_eval(params, save_predictions=False, model_name=None):
    look_back = params['look_back']
    X, y = make_sequences(scaled, look_back)
    split = int(len(X)*train_ratio)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    
    model = Sequential()
    model.add(LSTM(params['lstm_units'], input_shape=(look_back,1), kernel_regularizer=l2(params['l2_reg']), return_sequences=False))
    if params['dropout_rate']>0:
        model.add(Dropout(params['dropout_rate']))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mse')
    
    es = EarlyStopping(monitor='val_loss', patience=params['patience'], restore_best_weights=True)
    model.fit(X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], 
              validation_split=params['validation_split'], shuffle=params['shuffle'], 
              callbacks=[es], verbose=0)
    
    train_pred = model.predict(X_train, verbose=0)
    test_pred = model.predict(X_test, verbose=0)
    train_actual = scaler.inverse_transform(y_train)
    test_actual = scaler.inverse_transform(y_test)
    train_pred_actual = scaler.inverse_transform(train_pred)
    test_pred_actual = scaler.inverse_transform(test_pred)
    
    if save_predictions and model_name:
        train_dates = df['Date'].iloc[look_back:look_back+len(y_train)]
        test_dates = df['Date'].iloc[look_back+len(y_train):look_back+len(y_train)+len(y_test)]
        train_df = pd.DataFrame({'Date': train_dates, 'Actual': train_actual.flatten(), 'Predicted': train_pred_actual.flatten()})
        test_df = pd.DataFrame({'Date': test_dates, 'Actual': test_actual.flatten(), 'Predicted': test_pred_actual.flatten()})
        train_df.to_csv(f'data/processed/{model_name}_train_predictions.csv', index=False)
        test_df.to_csv(f'data/processed/{model_name}_test_predictions.csv', index=False)
    
    train_rmse = float(np.sqrt(mean_squared_error(train_actual, train_pred_actual)))
    train_mae = float(mean_absolute_error(train_actual, train_pred_actual))
    train_r2 = float(r2_score(train_actual, train_pred_actual))
    test_rmse = float(np.sqrt(mean_squared_error(test_actual, test_pred_actual)))
    test_mae = float(mean_absolute_error(test_actual, test_pred_actual))
    test_r2 = float(r2_score(test_actual, test_pred_actual))
    
    return {'train_rmse':train_rmse,'train_mae':train_mae,'train_r2':train_r2,
            'test_rmse':test_rmse,'test_mae':test_mae,'test_r2':test_r2}

## Baseline Model Evaluation

Establish baseline performance with default hyperparameters before optimization.

In [ ]:
baseline = {'look_back':20,'lstm_units':16,'dropout_rate':0.2,'l2_reg':0.0005,
           'learning_rate':0.001,'epochs':50,'batch_size':32,'patience':8,
           'validation_split':0.2,'shuffle':False}
before = train_eval(baseline, save_predictions=True, model_name='baseline')

## Optuna Objective Function

Define the optimization objective function with hyperparameter search spaces for Optuna.

In [ ]:
def objective(trial):
    params = {
        'look_back': trial.suggest_int('look_back', 15, 30),
        'lstm_units': trial.suggest_int('lstm_units', 16, 64),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.0, 0.4),
        'l2_reg': trial.suggest_float('l2_reg', 0.0, 0.001),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 5e-3, log=True),
        'epochs': trial.suggest_int('epochs', 30, 80),
        'batch_size': trial.suggest_categorical('batch_size', [16,32,64]),
        'patience': trial.suggest_int('patience', 5, 12),
        'validation_split': trial.suggest_float('validation_split', 0.1, 0.3),
        'shuffle': False
    }
    metrics = train_eval(params)
    return metrics['test_r2']

## Hyperparameter Optimization

Execute Optuna study to find optimal hyperparameters maximizing test R² score.

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=60, show_progress_bar=True)
best_params = study.best_params
best_params['shuffle']=False
metrics_after = train_eval(best_params, save_predictions=True, model_name='optimized')

## Results Comparison

Compare baseline and optimized model performance across all evaluation metrics.

In [ ]:
results = pd.DataFrame([dict(experiment='before',**before), 
                       dict(experiment='after',**metrics_after)])
results

## Actual vs Predicted Comparison: Before and After Hyperparameter Tuning

In [ ]:
baseline_train = pd.read_csv('data/processed/baseline_train_predictions.csv')
baseline_test = pd.read_csv('data/processed/baseline_test_predictions.csv')
optimized_train = pd.read_csv('data/processed/optimized_train_predictions.csv')
optimized_test = pd.read_csv('data/processed/optimized_test_predictions.csv')

baseline_train['Date'] = pd.to_datetime(baseline_train['Date'])
baseline_test['Date'] = pd.to_datetime(baseline_test['Date'])
optimized_train['Date'] = pd.to_datetime(optimized_train['Date'])
optimized_test['Date'] = pd.to_datetime(optimized_test['Date'])

os.makedirs('deliverables/hyperparameter_tuning_images', exist_ok=True)

fig, axes = plt.subplots(2, 2, figsize=(20, 12))
fig.suptitle('Actual vs Predicted: Before and After Hyperparameter Tuning', fontsize=16, fontweight='bold')

axes[0, 0].plot(baseline_train['Date'], baseline_train['Actual'], label='Actual', color='blue', alpha=0.7)
axes[0, 0].plot(baseline_train['Date'], baseline_train['Predicted'], label='Predicted', color='red', alpha=0.7)
axes[0, 0].set_title('Baseline Model - Training Set', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Price')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(baseline_test['Date'], baseline_test['Actual'], label='Actual', color='blue', alpha=0.7)
axes[0, 1].plot(baseline_test['Date'], baseline_test['Predicted'], label='Predicted', color='red', alpha=0.7)
axes[0, 1].set_title('Baseline Model - Test Set', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Price')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(optimized_train['Date'], optimized_train['Actual'], label='Actual', color='blue', alpha=0.7)
axes[1, 0].plot(optimized_train['Date'], optimized_train['Predicted'], label='Predicted', color='red', alpha=0.7)
axes[1, 0].set_title('Optimized Model - Training Set', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Price')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(optimized_test['Date'], optimized_test['Actual'], label='Actual', color='blue', alpha=0.7)
axes[1, 1].plot(optimized_test['Date'], optimized_test['Predicted'], label='Predicted', color='red', alpha=0.7)
axes[1, 1].set_title('Optimized Model - Test Set', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Price')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('deliverables/hyperparameter_tuning_images/before_after_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

baseline_combined = pd.concat([baseline_train, baseline_test]).sort_values('Date')
optimized_combined = pd.concat([optimized_train, optimized_test]).sort_values('Date')

fig, axes = plt.subplots(1, 2, figsize=(20, 6))
fig.suptitle('Full Timeline Comparison: Before vs After Hyperparameter Tuning', fontsize=16, fontweight='bold')

axes[0].plot(baseline_combined['Date'], baseline_combined['Actual'], label='Actual', color='blue', alpha=0.7)
axes[0].plot(baseline_combined['Date'], baseline_combined['Predicted'], label='Predicted', color='red', alpha=0.7)
axes[0].set_title('Baseline Model - Full Timeline', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(optimized_combined['Date'], optimized_combined['Actual'], label='Actual', color='blue', alpha=0.7)
axes[1].plot(optimized_combined['Date'], optimized_combined['Predicted'], label='Predicted', color='red', alpha=0.7)
axes[1].set_title('Optimized Model - Full Timeline', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('deliverables/hyperparameter_tuning_images/full_timeline_comparison.png', dpi=300, bbox_inches='tight')
plt.show()